# SciAgents
## Automating scientific discovery through multi-agent intelligent graph reasoning

#### Alireza Ghafarollahi, Markus J. Buehler, MIT, 2024 mbuehler@MIT.EDU

In [ ]:
!git clone https://github.com/lamm-mit/SciAgentsDiscovery.git
%cd SciAgentsDiscovery
!pip install -e .

In [1]:
import os

OpenAI_key='sk-'
os.environ['OPENAI_API_KEY']=OpenAI_key

data_dir_output='./graph_giant_component_LLMdiscovery_example/'

In [3]:
from ScienceDiscovery import *
make_dir_if_needed(data_dir_output)

'Directory already exists.'

### Setting up OpenAI GPT model for the LLM (non-automated version)

In [4]:
default_generate_OpenAIGPT = partial(
    generate_OpenAIGPT,
    openai_api_key=OpenAI_key,
    #gpt_model='gpt-4-turbo',
    gpt_model='gpt-4o',
    temperature=0.2,
    max_tokens=2048,
)

## 1- Research idea generation using the non-automated multi-agent model

In [ ]:
research_generation(G=G, 
                    embedding_tokenizer=embedding_tokenizer,
                    embedding_model=embedding_model,
                    node_embeddings=node_embeddings,
                    generate=default_generate_OpenAIGPT,
                    generate_graph_expansion=default_generate_OpenAIGPT,
                    randomness_factor=0.2, num_random_waypoints=4,shortest_path=False,
                    second_hop=False, data_dir=data_dir_output, save_files=False, verbatim=True,
                    keyword_1='energy-intensive', keyword_2='protein')

## 2- Research idea generation using the automated multi-agent model

In [ ]:
res = user.initiate_chat(recipient=manager,
message='''Develop a research proposal using random concepts. In the end, rate the novelty and feasibility of the research idea.''',
                        clear_history=True)

### Saving the output of the automated version

In [5]:
formatted_text = ""
formatted_text_summary = ""
for i in range(len(res.chat_history)):
    try:
        formatted_text += f'''{res.chat_history[i]['tool_calls'][0]['function']['name']}-{res.chat_history[1]['tool_calls'][0]['function']['arguments']}\n\n'''
    except:
        if i==0:
            formatted_text += '### ' + f'''{res.chat_history[i]['content']}\n\n'''
        else:
            formatted_text += f'''{res.chat_history[i]['content']}\n\n'''
            if re.search("Summary of the Initial Research Hypothesis", f'''{res.chat_history[i]['content']}'''):
                formatted_text_summary += f'''{res.chat_history[i]['content']}'''

text_markdown = Markdown(formatted_text)

markdown_to_pdf(formatted_text, 'output_research')